In [ ]:
from __future__ import division

In [ ]:
import csv
import pandas as pd
import datetime
import string
import numpy as np
import helper as hp
import sq
import jp
import urllib

In [ ]:
pd.set_option('display.max_columns', None)
Gb = 1000000000

In [ ]:
#store = pd.HDFStore('access_log.h5')

In [ ]:
def get_server_from_URL(URLs):
    result = []
    
    for u in URLs:
        if u.startswith('http://'):
            temp = u.split('http://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
        elif u.startswith('http://'):
            temp = u.split('https://')[1]
            # further split on '/' character to return only server 
            result.append(temp.split('/')[0])
        
    return result

In [ ]:
def remove_safe_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True
                break
                
        if (m == False):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result            

In [ ]:
def get_match_site(URLs, safe_list):
    result = []
    for u in URLs:
        m = False
        for s in safe_list:
            if s in u:
                m = True                
                break
                                
        if (m == True):
            result.append(u)
            
    result = pd.Series(result).unique()
        
    return result   

In [ ]:
#squid = sq.read_squid_log('access.log')

In [ ]:
#squid.tail()

In [ ]:
#safe_websites = 'google|microsoft|trendmicro|gstatic.com|bdpinsight.eu|.gov.hk'

In [ ]:
#access = squid[squid['URL'].str.contains(safe_websites) == False]

In [ ]:
#sum(squid.bytes)/Gb

In [ ]:
def juniper_log_cleansing(x):
    if str(x).find('=') > 0:
        return str(x)[str(x).find('=')+1:]
    elif str(x).find(':') > 0:
        return str(x)[:str(x).find(':')]    
    else:
        return x

def read_syslog_juniper(filename):
       
    df = hp.open_csv(filename)
    
    df = df[df[7].str.contains('information') == False]
    
    # restructure the dataframe
    i = [0]
    i.extend(range(3, 4))
    i.extend([1])
    i.extend(range(9, 13))
    i.extend([14])
    i.extend([16])
    i.extend(range(17, 25))
    df = df.iloc[:, i]
    
    df.columns = ['time', 'device_name', 'traffic_start', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port', 'remarks']
    df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']] = df[['device_name', 'duration', 'policy_no', 'service', 'protocol', 'src_zone', 'dst_zone', 'action', 'sent_size', 'received_size', 'src_address', 'dst_address', 'src_port', 'dst_port']].applymap(juniper_log_cleansing)
    
    return df

In [ ]:
infile = "C:\Temp\hk-ssg140.log-20150922"

In [ ]:
juniper = read_syslog_juniper(infile)

In [ ]:
np.sum(juniper[['sent_size', 'received_size']]) / Gb